**Store Loyalty System Implementation**

In [1]:
from abc import ABC, abstractmethod
from datetime import datetime
from typing import List


class Customer(ABC):
    """
    Abstract base class for all customers.
    Defines common attributes and methods for both member and non-member customers.
    """

    def __init__(self, customer_id: int):
        self.customer_id = customer_id
        self.invoices: List[Invoice] = []

    def add_invoice(self, invoice: 'Invoice') -> None:
        """Add an invoice to customer's history."""
        self.invoices.append(invoice)

    @abstractmethod
    def process_purchase(self, invoice: 'Invoice') -> str:
        """Process a purchase and handle any loyalty calculations."""
        pass


class Member(Customer):
    """
    Represents a member customer with loyalty points functionality.
    Members earn 1 point per $100 spent.
    """

    def __init__(self, customer_id: int):
        super().__init__(customer_id)
        self.loyalty_points = 0

    def process_purchase(self, invoice: 'Invoice') -> str:
        """
        Process purchase for member, calculating and adding loyalty points.
        Points are awarded at 1 point per $100 spent.
        """
        points_earned = int(
            invoice.amount // 100)  # Integer division for points
        self.loyalty_points += points_earned
        self.add_invoice(invoice)
        return f"Enter customer number: {self.customer_id}\nMember: Yes\nInvoice: {invoice.amount}\nPayment method: {invoice.payment_method.get_type()}\nPoints Earned: {points_earned}"


class NonMember(Customer):
    """
    Represents a non-member customer.
    Non-members don't earn loyalty points.
    """

    def process_purchase(self, invoice: 'Invoice') -> str:
        """Process purchase for non-member (no points calculation)."""
        self.add_invoice(invoice)
        return f"Enter customer number: {self.customer_id}\nMember: No\nInvoice: {invoice.amount}\nPayment method: {invoice.payment_method.get_type()}\nPoints Earned: 0"


class PaymentMethod(ABC):
    """Abstract base class for payment methods."""
    @abstractmethod
    def get_type(self) -> str:
        """Return the type of payment method."""
        pass


class CashPayment(PaymentMethod):
    """Represents cash payment method."""

    def get_type(self) -> str:
        return "Cash"


class CardPayment(PaymentMethod):
    """Represents card payment method."""

    def get_type(self) -> str:
        return "Card"


class Invoice:
    """
    Represents an invoice in the system.
    Stores amount, payment method, and date information.
    """

    def __init__(self, invoice_id: int, amount: float, payment_method: PaymentMethod):
        self.invoice_id = invoice_id
        self.amount = amount
        self.payment_method = payment_method
        self.date = datetime.now()


class StoreSystem:
    """
    Main system class that handles customer management and purchase processing.
    Acts as a facade for the entire system.
    """

    def __init__(self):
        self.customers: dict[int, Customer] = {}

    def register_customer(self, customer_id: int, is_member: bool) -> Customer:
        """Register a new customer in the system."""
        customer = Member(customer_id) if is_member else NonMember(customer_id)
        self.customers[customer_id] = customer
        return customer

    def process_purchase(self, customer_id: int, invoice_id: int,
                         amount: float, payment_type: str) -> str:
        """
        Process a purchase for a customer.
        Creates invoice and handles payment processing.
        """
        # Get or create payment method
        payment_method = CashPayment() if payment_type.lower() == "cash" else CardPayment()

        # Create invoice
        invoice = Invoice(invoice_id, amount, payment_method)

        # Process the purchase
        customer = self.customers.get(customer_id)
        if customer:
            return customer.process_purchase(invoice)
        else:
            raise ValueError(f"Customer {customer_id} not found")

# Example usage and test cases


def test_system():
    """Test the system with the provided test cases."""
    # Initialize system
    store = StoreSystem()

    # Test Case 1
    print("Case 1:")
    customer1 = store.register_customer(1111, True)  # Member
    result1 = store.process_purchase(1111, 1, 810, "Card")
    print(result1)

    # Test Case 2
    print("\nCase 2:")
    customer2 = store.register_customer(2222, False)  # Non-member
    result2 = store.process_purchase(2222, 2, 350, "Cash")
    print(result2)


if __name__ == "__main__":
    test_system()

Case 1:
Enter customer number: 1111
Member: Yes
Invoice: 810
Payment method: Card
Points Earned: 8

Case 2:
Enter customer number: 2222
Member: No
Invoice: 350
Payment method: Cash
Points Earned: 0


**Store System with Validation and Logging**

In [3]:
import logging
from datetime import datetime
from typing import Optional, Union
from dataclasses import dataclass

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='store_system.log'
)


@dataclass
class ValidationResult:
    """Data class to hold validation results."""
    is_valid: bool
    error_message: Optional[str] = None


class InputValidator:
    """Handles input validation for the store system."""

    @staticmethod
    def validate_customer_id(customer_id: int) -> ValidationResult:
        """Validate customer ID format and range."""
        if not isinstance(customer_id, int):
            return ValidationResult(False, "Customer ID must be an integer")
        if customer_id < 1000 or customer_id > 9999:
            return ValidationResult(False, "Customer ID must be between 1000 and 9999")
        return ValidationResult(True)

    @staticmethod
    def validate_amount(amount: float) -> ValidationResult:
        """Validate purchase amount."""
        if not isinstance(amount, (int, float)):
            return ValidationResult(False, "Amount must be a number")
        if amount <= 0:
            return ValidationResult(False, "Amount must be greater than 0")
        if amount > 1000000:  # Business rule: max transaction limit
            return ValidationResult(False, "Amount exceeds maximum transaction limit")
        return ValidationResult(True)

    @staticmethod
    def validate_payment_type(payment_type: str) -> ValidationResult:
        """Validate payment type."""
        valid_types = ["cash", "card"]
        if not isinstance(payment_type, str):
            return ValidationResult(False, "Payment type must be a string")
        if payment_type.lower() not in valid_types:
            return ValidationResult(False, f"Payment type must be one of {valid_types}")
        return ValidationResult(True)


class EnhancedStoreSystem(StoreSystem):
    """
    Enhanced version of StoreSystem with validation and logging.
    Inherits from original StoreSystem and adds additional functionality.
    """

    def __init__(self):
        super().__init__()
        self.validator = InputValidator()
        self.logger = logging.getLogger(__name__)

    def register_customer(self, customer_id: int, is_member: bool) -> Customer:
        """Register customer with validation."""
        # Validate input
        validation_result = self.validator.validate_customer_id(customer_id)
        if not validation_result.is_valid:
            self.logger.error(f"Customer registration failed: {
                              validation_result.error_message}")
            raise ValueError(validation_result.error_message)

        # Register customer
        try:
            customer = super().register_customer(customer_id, is_member)
            self.logger.info(f"Customer {customer_id} registered successfully as {
                             'member' if is_member else 'non-member'}")
            return customer
        except Exception as e:
            self.logger.error(
                f"Unexpected error during customer registration: {str(e)}")
            raise

    def process_purchase(self, customer_id: int, invoice_id: int,
                         amount: float, payment_type: str) -> str:
        """Process purchase with validation and logging."""
        # Validate all inputs
        validations = [
            self.validator.validate_customer_id(customer_id),
            self.validator.validate_amount(amount),
            self.validator.validate_payment_type(payment_type)
        ]

        for validation in validations:
            if not validation.is_valid:
                self.logger.error(f"Purchase processing failed: {
                                  validation.error_message}")
                raise ValueError(validation.error_message)

        try:
            result = super().process_purchase(customer_id, invoice_id, amount, payment_type)
            self.logger.info(f"Purchase processed successfully for customer {customer_id}, "
                             f"amount: {amount}, payment type: {payment_type}")
            return result
        except Exception as e:
            self.logger.error(f"Error processing purchase: {str(e)}")
            raise


def example_usage():
    """Example usage of enhanced system with validation and logging."""
    system = EnhancedStoreSystem()

    try:
        # Register customers
        system.register_customer(1111, True)
        system.register_customer(2222, False)

        # Process valid purchases
        print(system.process_purchase(1111, 1, 810, "Card"))
        print(system.process_purchase(2222, 2, 350, "Cash"))

        # Try invalid scenarios (will raise exceptions)
        # system.process_purchase(9999, 3, 100, "Card")  # Invalid customer ID
        # system.process_purchase(1111, 4, -100, "Card")  # Invalid amount
        # system.process_purchase(1111, 5, 100, "Bitcoin")  # Invalid payment type

    except ValueError as e:
        print(f"Validation error: {str(e)}")
    except Exception as e:
        print(f"Unexpected error: {str(e)}")


if __name__ == "__main__":
    example_usage()

Enter customer number: 1111
Member: Yes
Invoice: 810
Payment method: Card
Points Earned: 8
Enter customer number: 2222
Member: No
Invoice: 350
Payment method: Cash
Points Earned: 0
